# Loop 49 Strategic Analysis

## Critical Situation Assessment

After 50 experiments:
- Best CV: 0.008194 (exp_032)
- Best LB: 0.0877 (exp_030)
- Target: 0.0347
- Gap to target: 2.53x (0.0877 / 0.0347)
- Submissions remaining: 3

## Key Discovery: The "mixall" Kernel Uses GroupKFold(5)

The "mixall" kernel (8 votes, claims "good CV/LB") uses **GroupKFold(5)** instead of Leave-One-Out CV!

This is a CRITICAL difference:
- Our CV: Leave-One-Out (24 folds for single, 13 folds for full)
- mixall CV: GroupKFold(5) for both

**Hypothesis**: The hidden test evaluation might use a different CV scheme, and our CV-LB gap might be due to CV scheme mismatch.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load submission history
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},  # Best LB
    {'exp': 'exp_041', 'cv': 0.0090, 'lb': 0.0932},
    {'exp': 'exp_042', 'cv': 0.0145, 'lb': 0.1147},
]

df = pd.DataFrame(submissions)
print("Submission History:")
print(df)
print(f"\nBest LB: {df['lb'].min():.4f} from {df.loc[df['lb'].idxmin(), 'exp']}")
print(f"Best CV: {df['cv'].min():.4f} from {df.loc[df['cv'].idxmin(), 'exp']}")

In [ ]:
# Analyze CV-LB relationship
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])

print(f"CV-LB Relationship: LB = {slope:.2f}×CV + {intercept:.4f}")
print(f"R² = {r_value**2:.4f}")
print(f"\nIntercept: {intercept:.4f}")
print(f"Target: 0.0347")
print(f"\nCRITICAL: Intercept ({intercept:.4f}) > Target (0.0347)")
print(f"This means even with CV=0, predicted LB would be {intercept:.4f}")
print(f"\nTo reach target 0.0347, we need:")
required_cv = (0.0347 - intercept) / slope
print(f"Required CV = (0.0347 - {intercept:.4f}) / {slope:.2f} = {required_cv:.6f}")
print(f"\nThis is {'IMPOSSIBLE (negative CV)' if required_cv < 0 else 'POSSIBLE'}")

In [ ]:
# Plot CV-LB relationship
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], s=100, c='blue', alpha=0.7, label='Submissions')

# Regression line
cv_range = np.linspace(0, 0.02, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}×CV + {intercept:.4f}')

# Target line
plt.axhline(y=0.0347, color='green', linestyle=':', linewidth=2, label='Target (0.0347)')

# Best points
best_lb_idx = df['lb'].idxmin()
best_cv_idx = df['cv'].idxmin()
plt.scatter(df.loc[best_lb_idx, 'cv'], df.loc[best_lb_idx, 'lb'], s=200, c='red', marker='*', label=f'Best LB ({df.loc[best_lb_idx, "exp"]})')

plt.xlabel('CV Score')
plt.ylabel('LB Score')
plt.title('CV-LB Relationship After 50 Experiments')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print(f"\nGap to target: {df['lb'].min() / 0.0347:.2f}x")

In [ ]:
# Analyze what the "mixall" kernel does differently
print("="*70)
print("CRITICAL DISCOVERY: The 'mixall' kernel uses GroupKFold(5)")
print("="*70)
print()
print("Our CV scheme:")
print("  - Single solvent: Leave-One-Solvent-Out (24 folds, ~27 samples per fold)")
print("  - Full data: Leave-One-Ramp-Out (13 folds, ~94 samples per fold)")
print()
print("mixall CV scheme:")
print("  - Single solvent: GroupKFold(5) (~131 samples per fold)")
print("  - Full data: GroupKFold(5) (~245 samples per fold)")
print()
print("Key differences:")
print("  1. Leave-One-Out has HIGH VARIANCE (small test sets)")
print("  2. GroupKFold(5) has LOWER VARIANCE (larger test sets)")
print("  3. The hidden test evaluation might use a different CV scheme")
print()
print("HYPOTHESIS: The CV-LB gap might be due to CV scheme mismatch")
print("  - If hidden test uses GroupKFold(5), our Leave-One-Out CV is biased")
print("  - GroupKFold(5) might give a more accurate estimate of LB")

In [ ]:
# Analyze exp_042 (GroupKFold CV) - was it submitted?
print("="*70)
print("Checking exp_042 (GroupKFold CV)")
print("="*70)
print()
print("exp_042 used GroupKFold(5) CV and got:")
print("  - CV: 0.0145 (77% worse than best CV 0.008194)")
print("  - LB: 0.1147 (submitted)")
print()
print("This is ON THE SAME LINE as other submissions!")
print(f"  - Predicted LB using relationship: {slope * 0.0145 + intercept:.4f}")
print(f"  - Actual LB: 0.1147")
print(f"  - Ratio: {0.1147 / (slope * 0.0145 + intercept):.2f}x")
print()
print("CONCLUSION: GroupKFold(5) CV does NOT change the CV-LB relationship")
print("The gap is NOT due to CV scheme mismatch.")

In [ ]:
# What have we learned from 50 experiments?
print("="*70)
print("SUMMARY: What We've Learned from 50 Experiments")
print("="*70)
print()
print("1. ALL model families follow the SAME CV-LB relationship:")
print(f"   LB = {slope:.2f}×CV + {intercept:.4f} (R²={r_value**2:.4f})")
print()
print("2. Models tested:")
print("   - MLP (various architectures)")
print("   - LightGBM")
print("   - XGBoost")
print("   - RandomForest")
print("   - Ridge Regression")
print("   - Gaussian Process")
print("   - CatBoost")
print("   - k-NN")
print("   - Stacking")
print("   - Multi-Model Ensemble")
print()
print("3. Features tested:")
print("   - Spange descriptors")
print("   - DRFP (Differential Reaction Fingerprints)")
print("   - ACS PCA descriptors")
print("   - RDKit descriptors")
print("   - Fragprints")
print("   - Similarity features")
print("   - Arrhenius kinetics features")
print()
print("4. Techniques tested:")
print("   - Regularization (L1, L2, dropout)")
print("   - Ensemble averaging")
print("   - Stacking")
print("   - TTA (Test Time Augmentation)")
print("   - Per-target models")
print("   - Weighted loss")
print()
print("5. CV schemes tested:")
print("   - Leave-One-Out (default)")
print("   - GroupKFold(5)")
print()
print("NONE of these changed the CV-LB relationship!")

In [ ]:
# What could change the CV-LB relationship?
print("="*70)
print("WHAT COULD CHANGE THE CV-LB RELATIONSHIP?")
print("="*70)
print()
print("The intercept (0.0533) represents the 'baseline error' when extrapolating")
print("to unseen solvents. To reduce this, we need to:")
print()
print("1. FUNDAMENTALLY DIFFERENT APPROACH:")
print("   - Graph Neural Networks (GNN) - capture molecular structure")
print("   - The GNN benchmark achieved CV 0.0039 (5x better than our best)")
print("   - GNNs might have a DIFFERENT CV-LB relationship")
print()
print("2. PRE-TRAINED MOLECULAR EMBEDDINGS:")
print("   - Transfer learning from large-scale molecular datasets")
print("   - ChemBERTa, MolBERT, etc.")
print("   - These might capture generalizable molecular patterns")
print()
print("3. DOMAIN ADAPTATION:")
print("   - Identify what makes the hidden test different")
print("   - Adversarial validation to find drifting features")
print("   - Adjust model to be more robust")
print()
print("4. UNCERTAINTY-AWARE PREDICTIONS:")
print("   - Be more conservative on novel solvents")
print("   - Blend with a simple baseline when uncertainty is high")
print()
print("5. DIFFERENT EVALUATION METRIC:")
print("   - The hidden test might weight certain solvents differently")
print("   - We might be optimizing for the wrong thing")

In [ ]:
# Strategic recommendations
print("="*70)
print("STRATEGIC RECOMMENDATIONS (3 submissions remaining)")
print("="*70)
print()
print("Given that:")
print(f"  - Best CV: 0.008194 (exp_032, NOT submitted)")
print(f"  - Best LB: 0.0877 (exp_030)")
print(f"  - Target: 0.0347 (2.53x away)")
print(f"  - CV-LB relationship is STRUCTURAL (R²=0.981)")
print()
print("OPTION 1: Submit exp_032 (best CV)")
print(f"  - CV: 0.008194")
print(f"  - Predicted LB: {slope * 0.008194 + intercept:.4f}")
print(f"  - This would confirm the CV-LB relationship")
print()
print("OPTION 2: Try a fundamentally different approach (GNN)")
print("  - The GNN benchmark achieved CV 0.0039")
print("  - If GNN has a different CV-LB relationship, we might reach target")
print("  - BUT: GNN is complex and might not work in time")
print()
print("OPTION 3: Focus on reducing the intercept")
print("  - The intercept (0.0533) > target (0.0347)")
print("  - We need to find what causes the intercept")
print("  - Possible: uncertainty-aware predictions, domain adaptation")
print()
print("RECOMMENDATION:")
print("  1. Submit exp_032 (best CV) to confirm relationship")
print("  2. Try GNN or pre-trained embeddings")
print("  3. Final submission based on results")

In [ ]:
# Final analysis: What's the most promising path?
print("="*70)
print("FINAL ANALYSIS: Most Promising Path Forward")
print("="*70)
print()
print("After 50 experiments, we've established that:")
print("  1. The CV-LB relationship is STRUCTURAL (R²=0.981)")
print("  2. ALL model families follow the SAME line")
print("  3. The intercept (0.0533) > target (0.0347)")
print()
print("The ONLY way to reach the target is to:")
print("  A. Find an approach with a DIFFERENT CV-LB relationship")
print("  B. OR find that the target is unreachable (which we CANNOT accept)")
print()
print("Most promising approaches:")
print()
print("1. GRAPH NEURAL NETWORKS (GNN)")
print("   - The GNN benchmark achieved CV 0.0039")
print("   - GNNs capture molecular structure that tabular features miss")
print("   - This is the most likely path to a different CV-LB relationship")
print()
print("2. PRE-TRAINED MOLECULAR EMBEDDINGS")
print("   - ChemBERTa, MolBERT, etc.")
print("   - Transfer learning from large-scale datasets")
print("   - Might capture generalizable patterns")
print()
print("3. ENSEMBLE OF DIVERSE APPROACHES")
print("   - Combine GNN + MLP + LGBM")
print("   - Different approaches might have different biases")
print("   - The ensemble might have a lower intercept")
print()
print("CRITICAL: The target IS reachable. We just haven't found the right approach yet.")
print("The GNN benchmark proves that much better performance is possible.")

In [ ]:
# Check what the competition template says about evaluation
print("="*70)
print("COMPETITION EVALUATION NOTES")
print("="*70)
print()
print("From the competition description:")
print("  - Submissions are evaluated using cross-validation")
print("  - The last 3 cells must follow the template structure")
print("  - Only the model definition line can be changed")
print()
print("Key insight from the 'mixall' kernel:")
print("  - It uses GroupKFold(5) instead of Leave-One-Out")
print("  - It claims 'good CV/LB' with runtime of only 2m 15s")
print("  - It uses MLP + XGBoost + RF + LightGBM ensemble")
print()
print("BUT: Our exp_050 (Multi-Model Ensemble) got CV 0.009435")
print("This is 15% WORSE than our best CV (0.008194)")
print()
print("CONCLUSION: The 'mixall' approach doesn't help us.")
print("We need a fundamentally different approach.")

In [ ]:
# Summary of key findings
print("="*70)
print("KEY FINDINGS FOR NEXT ITERATION")
print("="*70)
print()
print("1. Multi-Model Ensemble (exp_050) FAILED:")
print("   - CV: 0.009435 (15% worse than best)")
print("   - Removing GP and adding RF/XGB hurt performance")
print("   - The GP component is VALUABLE")
print()
print("2. Best model remains GP(0.15) + MLP(0.55) + LGBM(0.3):")
print("   - exp_032: CV 0.008194 (best CV, NOT submitted)")
print("   - exp_030: CV 0.008298, LB 0.0877 (best LB)")
print()
print("3. CV-LB relationship is STRUCTURAL:")
print(f"   - LB = {slope:.2f}×CV + {intercept:.4f} (R²={r_value**2:.4f})")
print(f"   - Intercept ({intercept:.4f}) > Target (0.0347)")
print()
print("4. To reach target, we need:")
print("   - A fundamentally different approach (GNN, pre-trained embeddings)")
print("   - OR a way to reduce the intercept (uncertainty-aware predictions)")
print()
print("5. Submissions remaining: 3")
print("   - Use wisely to test fundamentally different approaches")
print("   - Don't waste on incremental improvements")